Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Rory Glenn"
COLLABORATORS = ""

---

# Homework 4: Sock Drawers and Arithmetic Dictionaries
## CSE 30 Fall 2020

### Copyright Luca de Alfaro, 2019-20.  License: CC-BY-NC. 

For how to work on this homework assignment, please refer to the instructions posted on Canvas. 

## Submission

[Please submit to this Google Form](https://forms.gle/UbDEiWqdfhSxzHnk7).

Deadline: Thursday October 22, 11pm (check on Canvas for updated information).

## About this homework
This homework notebook has many cells, as it is derived from the chapter, but there are only three questions. 
Each question is marked

    ### Question n:
    
for $n = 1, 2, 3$. 

## A Sock Drawer

Let us model a sock drawer.  How does a sock drawer work?  Well, when you do the washing, you end up with a bunch of socks, and you just put them in the drawer. So one operation is "put sock in drawer".  The other thing you typically do is look for two matching socks.  For simplicity, we will have only one attribute that we use to match socks: their color.  So the operations, in detail, are: 

* `add_sock`: adds a sock of a given color
* `get_pair`: ask if there is a pair of a given color, and if so, remove it
* `available_colors`: returns the set of colors for which there is at least a pair. 

Of course, you could also devise other operations, but these seem to be the basic ones.  

An aside: why does `available_colors` return a set and not a list? 
For two reasons.  First, returning a set makes clear (and indeed, enforces) that every color should appear only once in the return result. 
Second, in sets, the order of elements is irrelevant.  If we used a list, rather than a set, in any unit test suite we would have to: 

* Check that every element appears only once in the list; 
* sort the list of results before comparing it with the expected results, to ensure that different orders of elements in the returned list do not cause the test to fail. 

In general, it is quite important to choose as types for variables or return values, a type that expresses as best as possible the intended properties of the value.
Another thing: we want to define `available_colors` so that it is a _property_ rather than a method.  In this way, for a sock drawer `d`, we will be able to write `d.available_colors` rather than `d.available_colors()`.  It is a small difference, but in my opinion it does make the code look a little bit cleaner. 

What is a good implementation?  At first, one may consider simply implementing the drawer as a list of socks, where each sock is represented by its color.  This is simple enough at first:

In [ ]:
class SockDrawer(object):

    def __init__(self):
        self.drawer = []

    def add_sock(self, color):
        """Adds a sock of the given color to the drawer."""
        self.drawer.append(color)

    def get_pair(self, color):
        """Returns False if there is no pair of the given color, and True
        if there is.  In the latter case, it removes the pair from the drawer."""
        if self.drawer.count(color) >= 2:
            self.drawer.remove(color)
            self.drawer.remove(color)
            return True
        else:
            return False

    @property
    def available_colors(self):
        """Lists the colors for which we have at least two socks available."""
        colors = set()
        for el in self.drawer:
            if self.drawer.count(el) >= 2:
                colors.add(el)
        return colors


In the above implementation, `@property` is a [_decorator_](https://wiki.python.org/moin/PythonDecorators).  From the practical point of view, the [`@property` decorator](https://docs.python.org/3/library/functions.html#property) enables us to access the available colors of a sock drawer `sd` via `sd.available_colors` rather than `sd.available_colors()`: that is, it makes what is in truth a method call look like a variable access.  Big deal, you might say; all it does is it removes the need for parentheses!  True, but the eye also wants its part. 

In general, Python decorators are functions that take an input a function (the function just below them), and return another function; we will have occasion of seeing more of them later, and for the moment this short introduction shall suffice. 

Before we proceed further, let us write a function that puts the class through some tests.  We write this as a function taking the class as an argument (yes, in Python classes can be passed as arguments to functions).  In this way, when we define another (perhaps better) class that implements the sock drawer, we can test it using the same tests. 

**By popular request, we print Success! again.**

In [ ]:
#@title Let's define a testing helper.

def check_equal(x, y, msg=None):
    if x != y:
        if msg is None:
            print("Error:")
        else:
            print("Error in", msg, ":")
        print("    Your answer was:", x)
        print("    Correct answer: ", y)
    else:
        print("Success!")
    assert x == y, "%r and %r are different" % (x, y)


In [ ]:
def test_drawer(c):
    """Tests a drawer class c."""
    d = c()
    d.add_sock('red')
    d.add_sock('red')
    d.add_sock('red')
    d.add_sock('green')
    check_equal(d.available_colors, {'red'})
    check_equal(d.get_pair('red'), True)
    check_equal(d.get_pair('green'), False)
    check_equal(d.get_pair('red'), False)
    d.add_sock('blue')
    d.add_sock('blue')
    d.add_sock('blue')
    d.add_sock('red')
    check_equal(d.available_colors, {'red', 'blue'})


In [ ]:
test_drawer(SockDrawer)


Success!
Success!
Success!
Success!
Success!


I always find it a bit anticlimatic when all the tests pass, and all you get for this accomplishment is... nothing.  Yes, we could define a `PatOnTheBack` exception and raise it, but that will be for another time. 

Rather, let us discuss why the above implementation of a sock drawer is not great. 
A list keeps elements in a ... list.  So if you need to count the number of times in which an element occurs, you need to traverse the list.  The method count() is faster than the do-it-yourself solution `len([x for x in self.drawer if x == color])`,  as it is implemented natively, but it is still slow.  On top of that, once having counted the number of colors, if the chosen color appears more than once in `get_pair`, we call remove() _twice_ to remove it.  This is just esthetically terrible. We go over the list three times, each time unable to make use of the information we found the previous time. 

In the big scheme of things, the implementation might be fine.  "Going over the list" takes time proportional to the length of the list (it is a _linear-time_ operation), and unless a person has a very large number of socks, the performance may never become a bottleneck.  What is really disturbing, to me at least, is the ugliness of the code. 

## A Better Sock Drawer

Let us aim at a better solution.  The problem with our list-based implementation, fundamentally, is that all our questions can be answered by knowing how many socks we have for each color, and the list implementation stores this information in unary format, and mixing all colors together.  A much more rational way of remembering the status of the drawer is to associate with each color the number of socks we have.  Thus, our main data structure will be a dictionary mapping colors to integers. 

A short aside. If `d` is a normal dictionary, and you want to add `quantity` to `color`, you end up writing code like this:

In [ ]:
# Just to initialize
d = {'green': 4}
color = 'red'
quantity = 2

if color in d:
    d[color] += quantity
else:
    d[color] = quantity


Yes, you can do a little better, as in: 

In [ ]:
d[color] = d.get(color, 0) + quantity


but this is not nearly as elegant as being able to access a dictionary with square-bracket notation.  We would like some way of avoiding the test, or the concern, of whether a key is already in the dictionary.  

The [`defaultdict` collection type](https://docs.python.org/3/library/collections.html#collections.defaultdict) does precisely this.  A defaultdict dictionary, when accessed, returns the value associated with the key, if any, and otherwise returns a default value.  You specify the default value when you create the defaultdict, by passing to its creator a _factory_ function.  The factory function is called any time a default value is needed, and should return the default value.

In our case, we want a dictionary where the default value is 0; so we use the function `int` which, when called without arguments, returns 0.  Yes, it is a little confusing that `int` is both a type and a function... 

In [ ]:
int()


0

So we can create a dictionary with default value 0 as follows:

In [ ]:
from collections import defaultdict
d = defaultdict(int)

d['red'] += 2
print(d['red'])
print(d['green'])


2
0


With this we can define a better sock drawer:

In [ ]:
class SmartDrawer(object):

    def __init__(self):
        self.socks = defaultdict(int)

    def add_sock(self, color):
        self.socks[color] += 1

    def get_pair(self, color):
        if self.socks[color] > 1:
            self.socks[color] -= 2
            return True
        else:
            return False

    @property
    def available_colors(self):
        return {c for c, n in self.socks.items() if n > 1}


Note how this code is not only more efficient than our previous attempt, but also more concise.  Let us test this class.

In [ ]:
test_drawer(SmartDrawer)


Success!
Success!
Success!
Success!
Success!


# Arithmetic Dictionaries

At the core of our sock drawer implementation is a fundamental data structure which, for some reason, does not come standard in programming languages: a dictionary that has numerical values (and arbitrary keys). 

A number is a number.  An array, or vector, is a list of numbers that can be indexed by their position in the vector.  An _arithmetic dictionary_ is a set of numbers that can be addressed by their associated key.  Just as numbers, and arrays, can be combined via +, -, *, /, and compared with >, <, etc, so can arithmetic dictionaries.  Let us start defining them.  For brevity, we call them simply AD rather than ArithmeticDictionary.  We note that Python includes, in the `collections` module, the [`Counter` class](https://docs.python.org/3.7/library/collections.html#collections.Counter), which supports addition.  ADs are a more general version of the `Counter` class, as they support (or you can make them to support) all math operations you wish.

In Python, to be able to have two ADs `d1` and `d2`  and compute their sum via the syntax `d1 + d2`, [we need to define the `__add__` operation](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types), and similarly for the other arithmetic operations.  

There is a fine point to be discussed.  Arrays can be combined with arithmetical operations only when they are of the same dimensions, or when one can be "[broadcast](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html)" to fit the dimension of the other.  For dictionaries, however, the restriction to be able to combine only arithmetic dictionaries with the same keys would be too restrictive.  If we have an AD `d` representing a sock drawer, and we want to add to it one yellow and two blue socks, we want to be able to write something like: 

    d += AD({'yellow': 1, 'blue': 2})
    
rather than worry that `d` might have different keys from {'yellow', 'blue'}. Thus, in the definition of an operation, we will need to define suitable defaults for the missing values.  In general, the suitable defaults are the [_neutral elements_](https://en.wikipedia.org/wiki/Identity_element) with respect to the operation: 0 for $+$ and $-$, and 1 for $\times$ and /. 

Let us begin from addition and subtraction.

In [ ]:
class AD(dict):

    def __init__(self, *args, **kwargs):
        """This initializer simply passes all arguments to dict, so that
        we can create an AD with the same ease with which we can create
        a dict.  There is no need, indeed, to repeat the initializer,
        but we leave it here in case we like to create attributes specific
        of an AD later."""
        super().__init__(*args, **kwargs)

    def __add__(self, other):
        return AD._binary_op(self, other, lambda x, y: x + y, 0)

    def __sub__(self, other):
        return AD._binary_op(self, other, lambda x, y: x - y, 0)

    @staticmethod
    def _binary_op(left, right, op, neutral):
        r = AD()
        l_keys = set(left.keys()) if isinstance(left, dict) else set()
        r_keys = set(right.keys()) if isinstance(right, dict) else set()
        for k in l_keys | r_keys:
            # If the right (or left) element is a dictionary (or an AD),
            # we get the elements from the dictionary; else we use the right
            # or left value itself.  This implements a sort of dictionary
            # broadcasting.
            l_val = left.get(k, neutral) if isinstance(left, dict) else left
            r_val = right.get(k, neutral) if isinstance(right, dict) else right
            r[k] = op(l_val, r_val)
        return r



In the implementation above, we have factored together what would have been the common part of the implementation of $+$ and $-$ in the method `_binary_op`.  The `_binary_op` method is a [_static method:_](https://docs.python.org/3/library/functions.html#staticmethod) it does not refer to a specific object.  In this way, we can decide separately what to provide to it as the left and right hand sides; this will be useful later, as we shall see.  To call it, we have to refer to it via `AD._binary_op` rather than `self._binary_op`, as would be the case for a normal (object) method.  Furthermore, in the `_binary_op` method, we do not have access to `self`, since `self` refers to a specific object, while the method is generic, defined for the class. 

The `_binary_op` method combines values from its `left` and `right` operands using operator `op`, and the default `neutral` if a value is not found.  The two operands `left` and `right` are interpreted as dictionaries if they are a subclass of `dict`, and are interpreted as constants otherwise.

We have defined our class AD as a _subclass_ of the dictionary class `dict`.  This enables us to apply to an AD all the [methods that are defined for a dictionary](https://docs.python.org/3/library/stdtypes.html#typesmapping), which is really quite handy. 

In [ ]:
d1 = AD()
d1['red'] = 2
print(d1)
d2 = AD({'blue': 4, 'green': 5})
print(d2)


{'red': 2}
{'blue': 4, 'green': 5}


In the code above, note how we can write `d1['red']`: our ability to access ADs via the square-bracket notation is due to the fact that ADs, like dictionaries, implement the [`__getitem__` method](https://docs.python.org/3/reference/datamodel.html#object.__getitem__).  Note also that AD inherits from `dict` even the initializer methods, so that we can pass to our dictionary another dictionary, and obtain an AD that contains a [_copy_ of that dictionary](https://docs.python.org/3/library/stdtypes.html#dict). 

We can see that addition and subtraction also work:

In [ ]:
print(AD(red=2, green=3) + AD(red=1, blue=4))
print(AD(red=2, green=3) - AD(red=1, blue=4))


{'green': 3, 'red': 3, 'blue': 4}
{'green': 3, 'red': 1, 'blue': -4}


We let multiplication and division to you to implement.
You need to implement the operators:

* `__mul__`
* `__truediv__`
* `__floordiv__`

In [ ]:
### Question 1: Implement multiplication and division

# Define __mul__
def ad_mul(self, other):
  return AD._binary_op(self, other, lambda x, y: x * y, 1)

AD.__mul__ = ad_mul

# Define below __truediv__, similarly.
def ad_truediv(self, other):
  return AD._binary_op(self, other, lambda x, y: float(x) / float(y), 1)

AD.__truediv__ = ad_truediv

def ad_floordiv(self, other):
  return AD._binary_op(self, other, lambda x, y: x // y, 1)

AD.__floordiv__ = ad_floordiv


In [ ]:
### Tests for multiplication

# Note that the elements that are missing in the other dictionary
# are assumed to be 1.
d = AD(a=2, b=3) * AD(b=4, c=5)
check_equal(d, AD(a=2, b=12, c=5))



Success!


In [ ]:
### Tests for division

d = AD(a=8, b=6) / AD(a=2, c=4)
check_equal(d, AD(a=4, b=6, c=0.25))



Success!


In [ ]:
### Tests for integer division

d = AD(a=8, b=6) // AD(a=2, b=4, c=4)
check_equal(d, AD(a=4, b=1, c=0))



Success!


We will let you also implement max_items and min_items properties.  The value of these properties should be a _pair_, consisting of the maximum/minimum value, and of the _set_ of keys that led to that value.  The latter needs to be a set, because there are multiple key, value pairs where the value is maximal or minimal. To avoid wasting your time, we will have you implement only the max_items property.  If the AD is empty, the property's value should be the pair `(None, set())`. 

In [ ]:
### Question 2: Implement `max_items`

def ad_max_items(self):
    """Returns a pair consisting of the max value in the AD, and of the
    set of keys that attain that value. This can be done in 7 lines of code."""
    maximum = 0
    s = set()

    for key, value in self.items():
      if value > maximum:
        s = set()
        s.add(key)
        maximum = value
      elif value == maximum:
        s.add(key)
    
    if maximum == 0:
      return None, set()
    
    return maximum, s

# Remember the use of the property decorator.
AD.max_items = property(ad_max_items)


In [ ]:
### Tests for `max_items`

# For empty ADs, it has to return None as value, and the empty set as key set.
check_equal(AD().max_items, (None, set()))
# Case of one maximum.
check_equal(AD(red=2, green=3, blue=1).max_items, (3, {'green'}))
# Case of multiple maxima.
check_equal(AD(red=2, yellow=3, blue=3, violet=3, pink=1).max_items,
             (3, {'yellow', 'blue', 'violet'}))


Success!
Success!
Success!


## Comparison and Logical Operators

### Comparisons

If `d` is an `AD`, what is the meaning of `d > 2`?

In numpy, when we have an array `a`, and we write `a > 2`, we obtain the array consisting of boolean entries, indicating whether the elements of `a` are greater than two or not: 


In [ ]:
import numpy as np

a = np.array([2, 3, 4, 1, 2])
a > 2


array([False,  True,  True, False, False])

This suggests adopting a similar semantics for inequalities, defining the behavior of the [comparison operators](https://docs.python.org/3/reference/datamodel.html#object.__lt__):

In [ ]:
AD.__lt__ = lambda l, r: AD._binary_op(l, r, lambda x, y: x < y, 0)
AD.__gt__ = lambda l, r: AD._binary_op(l, r, lambda x, y: x > y, 0)
AD.__le__ = lambda l, r: AD._binary_op(l, r, lambda x, y: x <= y, 0)
AD.__ge__ = lambda l, r: AD._binary_op(l, r, lambda x, y: x >= y, 0)


Let us see how this works: 

In [ ]:
AD(cat=4, bird=2) > 2


{'bird': False, 'cat': True}

Once we have a dictionary mapping from values to booleans, we need some way of testing whether all keys, or some keys, map to True. In Python, the `all()` function returns whether all elements in a boolean list are true: 

In [ ]:
print(all([True, False, True, True]))
print(all([True, True, True, True]))


False
True


So we can define the `all` and `any` methods for an AD as follows: 

In [ ]:
AD.all = lambda self : all(self.values())
AD.any = lambda self : any(self.values())


We also add a method for filtering the elements that satisfy a condition.  If the condition is left unspecified, we use the truth-value of the elements themselves.

In [ ]:
def ad_filter(self, f=None):
    return AD({k: v for k, v in self.items() if (f(v) if f is not None else v)})
AD.filter = ad_filter


Let us try this out.

In [ ]:
d = AD(green=3, blue=2, red=1)
d.filter(lambda x : x > 1)


{'blue': 2, 'green': 3}

In [ ]:
d = AD(green=3, blue=2, red=1)
(d > 1).filter()


{'blue': True, 'green': True}

In [ ]:
(d > 1).all()


False

In [ ]:
(d > 0).all()


True

## Implementing Bag of Words in Terms of Arithmetic Dictionaries

In natural language processing, a common (if not very detailed!) representation for a piece of text is the (_Bag of Words_)[https://en.wikipedia.org/wiki/Bag-of-words_model] model. 
It's quite simple: we count how many times a word appears in a piece of text, and that's it! 
We can implement bag of words quite simply in terms of our Arithmetic Dictionaries. 

We want to create a bag of words via: 

    bow = BoW("some piece of text")

We then want all the operations we defined for ADs to work also for bags of words.  For instance, we want to be able to do: 

    bow1 = BoW("I like cakes")
    bow2 = BoW("I drink coffee")
    bow3 = bow1 + bow2

and we want to get as result the bag of words corresponding to the count:

    {"I": 2, "like": 1, "cakes": 1, "drink": 1, "coffee": 1}


## A pedestrian solution: a class with an AD member

There are two ways of doing this. 
The most pedestrian is to define a new class BaW (note the intentional mis-spelling), with an AD as a member. 

In [ ]:
class BaW(object):

    def __init__(self, text=None):
        # Here, you have to split the text, and define self.ad to
        # be an AD corresponding to the text.
        # It can be done in 4 lines of code.
        self.ad =  AD()
        if text is not None:
            for w in text.split():
                self.ad += AD({w: 1})

    def __add__(self, other):
        b = BaW()
        b.ad = self.ad + other.ad
        return b

    def __repr__(self):
        return repr(self.ad)


In [ ]:
baw1 = BaW("I like to pick raisins")
baw2 = BaW("I do not like to pick apples")
print(baw1 + baw2)
print(baw1)


{'apples': 1, 'not': 1, 'I': 2, 'do': 1, 'raisins': 1, 'to': 2, 'pick': 2, 'like': 2}
{'raisins': 1, 'to': 1, 'pick': 1, 'I': 1, 'like': 1}


In [ ]:
### Tests for BaW
baw1 = BaW("I like to pick raisins")
baw2 = BaW("I do not like to pick pick apples")
baw3 = baw1 + baw2
check_equal(baw1.ad['like'], 1)
check_equal(baw3.ad['like'], 2)
check_equal(baw3.ad['pick'], 3)

Success!
Success!
Success!


## A sophisticated solution: subclassing AD

Defining a new class BaW however is a big waste. Then we have to redefine all the methods we need, from `__add__`, to `__sub__`, to the inequalities... 
In truth, all we need to do is make BoW (now spelled correctly) a subclass of AD, replacing its initializer so it can take a piece of text rather than a dictionary initializer. 

Replacing the initializer is slightly tricky, so we will help you do it.  First, the initializer calls the superclass initializer, like so:

    def __init__(self, text):
        super().__init__()

Next, you compute an AD `ad` representing the word occurrences in `text`, just as you did for BaW. 

Then, there are two ways to copy `ad` into `self`: one is to do:

    self.update(ad)

and the other is to do:

    for k, v in ad.items():
        self[k] = v

Choose whichever way you want.

### A third way

There is also another way of writing the initializer for BoW, which we mention here for the sake of completeness.  
In this third way, you first compute the AD `ad`, and _then_ you call the superclass initializer with `ad` as an argument. 

    def __init__(self, text):
        ad = ... # Compute it somehow
        super().__init__(ad)

Do not use this way in your solution: in your solution, the superclass initializer is called at the beginning, and calling it twice won't work.


In [ ]:
### Question 3: Define BoW (you need to write only the initializer).

class BoW(AD):

  def __init__(self, text):
    self.ad =  AD()
    if text is not None:
      for w in text.split():
        self.ad += AD({w: 1})
    super().__init__(self.ad)

The beauty of subclassing is that we only have to replace the initializer: all the implementations of `__add__`, etc, are already there for us to use.

In [ ]:
bow1 = BoW("I like to eat cakes to go")
bow2 = BoW("I like to drink coffee")
bow1 + bow2

{'I': 2,
 'cakes': 1,
 'coffee': 1,
 'drink': 1,
 'eat': 1,
 'go': 1,
 'like': 2,
 'to': 3}

In [ ]:
bow1 - bow2


{'I': 0,
 'cakes': 1,
 'coffee': -1,
 'drink': -1,
 'eat': 1,
 'go': 1,
 'like': 0,
 'to': 1}

In [ ]:
### Tests for BoW, base case. 

empty_bow = BoW("")
check_equal(len(empty_bow), 0)

Success!


In [ ]:
### We check that you really build an AD.

simple_bow = BoW("I like apples")
check_equal(isinstance(simple_bow, AD), True)
check_equal(set(simple_bow.keys()), {'I', 'like', 'apples'})


Success!
Success!


In [ ]:
### Tests for BoW, full behavior. 

bow1 = BoW("I like to eat cakes to go")
bow2 = BoW("I like to drink coffee")
bow3 = bow1 + bow2
check_equal(bow3['I'], 2)
check_equal(bow3['to'], 3)



Success!
Success!
